In [150]:
import pandas as pd
from pytz import country_timezones
import time
import numpy as np
import timezonefinder
from datetime import datetime as dt
import pytz
import calendar

In [4]:
data = pd.read_csv("C:/Users/fabri/OneDrive/Desktop/GIT/Data_Management_Project/Csv/Exported_queries/mtime.csv") #import dataset
data.head() #check

,mtime,group_country,group_state,long,lat
0,1545782699258,us,FL,-80.15,26.02
1,1545827677229,us,PA,-75.20,39.96
2,1545785377241,nz,NONE,174.78,-41.28
3,1545780008634,us,NY,-73.38,40.69
4,1545770541536,us,TX,-106.49,31.76


In [5]:
data.group_state = data.group_state.str.lower() #lower case for group_state
data.head() #check

,mtime,group_country,group_state,long,lat
0,1545782699258,us,fl,-80.15,26.02
1,1545827677229,us,pa,-75.20,39.96
2,1545785377241,nz,none,174.78,-41.28
3,1545780008634,us,ny,-73.38,40.69
4,1545770541536,us,tx,-106.49,31.76


In [6]:
tf = timezonefinder.TimezoneFinder() #defining time zones on lat, long data
data['timezone_str'] = "nan"
count = 0
start = time.time()
for line in data.itertuples():
    count+=1
    try:
        data.timezone_str.at[line.Index] = tf.timezone_at(lat = data.iloc[line.Index]['lat'],
                                                          lng = data.iloc[line.Index]['long'])
    except Exception as e:
        print(e)
        break
    #if count == 10:
        #break
end = time.time()
print("task completed in "+str((end-start)/60)+" minutes")

task completed in 11.484340254465739 minutes


In [9]:
data.head() #check

,mtime,group_country,group_state,long,lat,timezone_str
0,1545782699258,us,fl,-80.15,26.02,America/New_York
1,1545827677229,us,pa,-75.20,39.96,America/New_York
2,1545785377241,nz,none,174.78,-41.28,Pacific/Auckland
3,1545780008634,us,ny,-73.38,40.69,America/New_York
4,1545770541536,us,tx,-106.49,31.76,America/Denver


In [10]:
data.any().isnull() #check

mtime            False
group_country    False
group_state      False
long             False
lat              False
timezone_str     False
dtype: bool

In [133]:
data1 = data[data.lat != 0.0] #cleaned dataset
data1[data1.lat == 0] #check

,mtime,group_country,group_state,long,lat,timezone_str


In [12]:
data1.any().isnull() #another check, no more missing values

mtime            False
group_country    False
group_state      False
long             False
lat              False
timezone_str     False
dtype: bool

In [138]:
def rounder(t): #this function round the time to nearest hour
    if t.minute >= 30:
        if t.hour == 23:
            return t.replace(second=0, microsecond=0, minute=0, hour=0)
        else:
            return t.replace(second=0, microsecond=0, minute=0, hour= t.hour+1)
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [139]:
#data1['datetime_timezoned'] = "nan" #this method create problems with "SettingWithCopyWarning" don't know why
data1 = data1.assign(datetime_timezoned = "") #defining new column converting timestamp into datetime conditionally to time zones
start = time.time()
count = 0
for line in data1.itertuples():
    count+=1
    try: #this first line is to eventually create a column with the original UTC datetime format
        #data1.loc[line.Index, 'datetime'] = dt.fromtimestamp(timestamp = (data1.loc[line.Index, 'mtime'])/1000)
        data1.datetime_timezoned.at[line.Index] = rounder((dt.fromtimestamp
                                                   (timestamp = (data1.loc[line.Index, 'mtime'])/1000,
                                                    tz = pytz.timezone(data1.loc[line.Index, 'timezone_str'])
                                                   ))).strftime("%d/%m/%Y %H:%M:%S")
        #data1.datetime_timezoned.at[line.Index] = data1.iloc[line.Index]['datetime_timezoned'].strftime("%d/%m/%Y %H:%M:%S")
        #these two lines create first of all a column with the datetime timezoned based on the time zone of the group that
        #host the event, and in second time they change the format of that datetime
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')

task completed in 2.862696619828542 minutes


In [140]:
data1.head()

,mtime,group_country,group_state,long,lat,timezone_str,datetime_timezoned
0,1545782699258,us,fl,-80.15,26.02,America/New_York,25/12/2018 19:00:00
1,1545827677229,us,pa,-75.20,39.96,America/New_York,26/12/2018 08:00:00
2,1545785377241,nz,none,174.78,-41.28,Pacific/Auckland,26/12/2018 14:00:00
3,1545780008634,us,ny,-73.38,40.69,America/New_York,25/12/2018 18:00:00
4,1545770541536,us,tx,-106.49,31.76,America/Denver,25/12/2018 14:00:00


In [107]:
#this code use a list to create the datetime_timezoned column, more efficient, 1 minute less than the other method
'''
datetime_timezoned = []
start = time.time()
count = 0
for line in data1.itertuples():
    count+=1
    try: #this first line is to eventually create a column with the original UTC datetime format
        #data1.loc[line.Index, 'datetime'] = dt.fromtimestamp(timestamp = (data1.loc[line.Index, 'mtime'])/1000)
        datetime_timezoned.append((dt.fromtimestamp(timestamp = (data1.loc[line.Index, 'mtime'])/1000,
                                                    tz = pytz.timezone(data1.loc[line.Index, 'timezone_str'])
                                                   )).strftime("%d/%m/%Y %H:%M:%S"))
        #data1.datetime_timezoned.at[line.Index] = data1.iloc[line.Index]['datetime_timezoned'].strftime("%d/%m/%Y %H:%M:%S")
        #these two lines create first of all a column with the datetime timezoned based on the time zone of the group that
        #host the event, and in second time they change the format of that datetime
    except Exception as e:
        print(e)
        break
    #if count == 10000:
        #break
data1['datetime_timezoned'] = datetime_timezoned
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')
'''

task completed in 1.9935708483060202 minutes


In [141]:
new = data1['datetime_timezoned'].str.split(" ", n = 1, expand = True) #create distinct columns date and time
data1['date'] = [calendar.day_name[dt.strptime(date, '%d/%m/%Y').weekday()] for date in new[0]] #transform date into weekdays
data1['time'] = new[1] #take time

In [159]:
data1.head() #check

,mtime,group_country,group_state,long,lat,timezone_str,datetime_timezoned,date,time
0,1545782699258,us,fl,-80.15,26.02,America/New_York,25/12/2018 19:00:00,Tuesday,19:00:00
1,1545827677229,us,pa,-75.20,39.96,America/New_York,26/12/2018 08:00:00,Wednesday,08:00:00
2,1545785377241,nz,none,174.78,-41.28,Pacific/Auckland,26/12/2018 14:00:00,Wednesday,14:00:00
3,1545780008634,us,ny,-73.38,40.69,America/New_York,25/12/2018 18:00:00,Tuesday,18:00:00
4,1545770541536,us,tx,-106.49,31.76,America/Denver,25/12/2018 14:00:00,Tuesday,14:00:00
